In [7]:
from dataclasses import dataclass, field
from typing import List, Dict, Tuple, Iterator
from itertools import product
from prettyprinter import pprint

@dataclass(frozen=True)
class Variable:
    """
    Represents a random variable in a Bayesian network.

    Attributes:
        name (str): The name of the variable.
        r (int): The number of possible values (states) the variable can take.
    """
    name: str
    r: int  # Number of possible values
    
    def __repr__(self):
        return f"{self.name}"        

@dataclass(frozen=True)
class Assignment:
    """
    Represents an assignment of values to variables.

    Attributes:
        values (Dict[str, int]): A mapping from variable names to their assigned values.
    """
    values: Dict[str, int]

    def __getitem__(self, var_name: str) -> int:
        return self.values[var_name]

    def __repr__(self):
        return f"Assignment({self.values})"
    
    
    def __hash__(self):
        # Use frozenset of items to make it hashable
        return hash(frozenset(self.values.items()))

    def __eq__(self, other):
        if not isinstance(other, Assignment):
            return NotImplemented
        return self.values == other.values

@dataclass
class Factor:
    """
    Represents a factor in the Bayesian network.

    Attributes:
        variables (List[Variable]): Variables involved in the factor.
        table (Dict[Assignment, float]): A mapping from variable assignments to probabilities.
    """
    variables: List[Variable]
    table: Dict[Assignment, float] = field(default_factory=dict)

    def __post_init__(self):
        self.var_names = [var.name for var in self.variables]

    def assignments(self) -> List[Assignment]:
        """
        Generates all possible assignments for the variables in this factor.

        Returns:
            List[Assignment]: A list of all possible assignments.
        """
        ranges = [range(1, var.r + 1) for var in self.variables]
        assignments = []
        for values in product(*ranges):
            assignment = Assignment(values=dict(zip(self.var_names, values)))
            assignments.append(assignment)
        return assignments

    def normalize(self):
        """
        Normalizes the factor so that the sum of probabilities is 1.
        """
        total = sum(self.table.values())
        if total != 0:
            for assignment in self.table:
                self.table[assignment] /= total

    def marginalize(self, var_to_marginalize: Variable) -> 'Factor':
        """
        Marginalizes out a variable from the factor.

        Args:
            var_to_marginalize (Variable): The variable to be marginalized out.

        Returns:
            Factor: A new factor with the variable marginalized out.
        """
        new_vars = [var for var in self.variables if var != var_to_marginalize]
        new_table = {}
        for assignment in self.assignments():
            # Remove the variable to marginalize from the assignment
            new_assignment_values = {k: v for k, v in assignment.values.items() if k != var_to_marginalize.name}
            new_assignment = Assignment(new_assignment_values)
            prob = self.table.get(assignment, 0.0)
            # Sum probabilities for the same assignment after removing the variable
            new_table[new_assignment] = new_table.get(new_assignment, 0.0) + prob
        return Factor(new_vars, new_table)

    def __mul__(self, other: 'Factor') -> 'Factor':
        """
        Multiplies this factor with another factor.

        Args:
            other (Factor): The other factor to multiply with.

        Returns:
            Factor: A new factor resulting from the multiplication.
        """
        # Determine the set of variables in the new factor
        new_vars_dict = {var.name: var for var in self.variables + other.variables}
        new_vars = list(new_vars_dict.values())
        new_var_names = [var.name for var in new_vars]

        # Prepare the table for the new factor
        new_table = {}

        # Generate all possible assignments for new_vars
        ranges = [range(1, var.r + 1) for var in new_vars]
        for values in product(*ranges):
            assignment_values = dict(zip(new_var_names, values))
            assignment = Assignment(assignment_values)

            # Get the values from self and other for this assignment
            self_assignment_values = {k: v for k, v in assignment_values.items() if k in [var.name for var in self.variables]}
            self_assignment = Assignment(self_assignment_values)
            self_value = self.table.get(self_assignment, 0.0)

            other_assignment_values = {k: v for k, v in assignment_values.items() if k in [var.name for var in other.variables]}
            other_assignment = Assignment(other_assignment_values)
            other_value = other.table.get(other_assignment, 0.0)

            # Multiply the values
            new_table[assignment] = self_value * other_value

        return Factor(new_vars, new_table)

    def __repr__(self):
        table_str = "\n".join([str(item) for item in self.table.items()])
        return f"Factor(variables={self.var_names}, table=\n{table_str})"

# Example usage:

# Define variables
x = Variable("x", 2)
y = Variable("y", 2)
z = Variable("z", 2)


# Define a factor table
ft = {
    Assignment(values={'x':1,'y':1,'z':1}):64, 
    Assignment(values={'x':1,'y':1,'z':2}):32,
    Assignment(values={'x':1,'y':2,'z':1}):16, 
    Assignment(values={'x':1,'y':2,'z':2}):8,
    Assignment(values={'x':2,'y':1,'z':1}):4, 
    Assignment(values={'x':2,'y':1,'z':2}):2,
    Assignment(values={'x':2,'y':2,'z':1}):1, 
    Assignment(values={'x':2,'y':2,'z':2}):1,
}

# Create a factor
phi = Factor(variables=[x, y, z], table=ft)

print(f'phi:\n{phi}')


print('-' * 100)


# Normalize the factor
phi.normalize()
pprint(phi, indent=2)

# Marginalize out variable 'z'
phi_marginalized = phi.marginalize(z)

pprint(phi_marginalized, indent=2)
# Multiply two factors (assuming another factor 'psi' is defined)

psi = Factor(variables=[x, y, z], table = {
    Assignment(values={'x':1,'y':1,'z':1}):-1, 
    Assignment(values={'x':1,'y':1,'z':2}):-1,
    Assignment(values={'x':1,'y':2,'z':1}):-1, 
    Assignment(values={'x':1,'y':2,'z':2}):10,
    Assignment(values={'x':2,'y':1,'z':1}):-1, 
    Assignment(values={'x':2,'y':1,'z':2}):-1,
    Assignment(values={'x':2,'y':2,'z':1}):-1, 
    Assignment(values={'x':2,'y':2,'z':2}):-1,
})


phi_psi = phi * psi

pprint(phi_psi)


phi:
Factor(variables=['x', 'y', 'z'], table=
(Assignment({'x': 1, 'y': 1, 'z': 1}), 64)
(Assignment({'x': 1, 'y': 1, 'z': 2}), 32)
(Assignment({'x': 1, 'y': 2, 'z': 1}), 16)
(Assignment({'x': 1, 'y': 2, 'z': 2}), 8)
(Assignment({'x': 2, 'y': 1, 'z': 1}), 4)
(Assignment({'x': 2, 'y': 1, 'z': 2}), 2)
(Assignment({'x': 2, 'y': 2, 'z': 1}), 1)
(Assignment({'x': 2, 'y': 2, 'z': 2}), 1))
----------------------------------------------------------------------------------------------------
Factor(variables=['x', 'y', 'z'], table=
(Assignment({'x': 1, 'y': 1, 'z': 1}), 0.5)
(Assignment({'x': 1, 'y': 1, 'z': 2}), 0.25)
(Assignment({'x': 1, 'y': 2, 'z': 1}), 0.125)
(Assignment({'x': 1, 'y': 2, 'z': 2}), 0.0625)
(Assignment({'x': 2, 'y': 1, 'z': 1}), 0.03125)
(Assignment({'x': 2, 'y': 1, 'z': 2}), 0.015625)
(Assignment({'x': 2, 'y': 2, 'z': 1}), 0.0078125)
(Assignment({'x': 2, 'y': 2, 'z': 2}), 0.0078125))
Factor(variables=['x', 'y'], table=
(Assignment({'x': 1, 'y': 1}), 0.75)
(Assignment({'x': 1

# Your goal is to:Define the variables and their possible values.
1) Create the conditional probability tables (CPTs) for each variable.
2) Construct the Bayesian Network using the provided classes.
3) Given evidence (e.g., a positive test result), compute the posterior probability of the patient having the disease.
4) Analyze the results and discuss the implications in decision-making.

# #2 Define Variables


In [4]:
D = Variable("D", 2)  # Disease: 1 (Yes), 2 (No)
T = Variable("T", 2) # Test Result: 1 (Positive), 2 (Negative) 
S = Variable("S", 2) # Symptom: 1 (Present), 2 (Absent)

# 3. Define Probability Tables
Create the conditional probability tables for:

P(D): Prior probability of the disease.

P(T | D): Probability of the test result given the disease.

P(S | D): Probability of the symptom given the disease.

Assume the following probabilities:

P(D):

P(D=1) = 0.01 (1% prevalence)

P(D=2) = 0.99


P(T | D):

If D=1 (Disease present):

P(T=1 | D=1) = 0.95 (True Positive Rate)

P(T=2 | D=1) = 0.05

If D=2 (Disease absent):

P(T=1 | D=2) = 0.10 (False Positive Rate)

P(T=2 | D=2) = 0.90

P(S | D):

If D=1:

P(S=1 | D=1) = 0.80

P(S=2 | D=1) = 0.20

If D=2:

P(S=1 | D=2) = 0.30

P(S=2 | D=2) = 0.70

In [39]:
var = {
    'P(D)': {
        Assignment(values=dict(D=1)): 0.01,
        Assignment(values=dict(D=2)): 0.99,
    },
    
    'P(T|D)': {
        Assignment(values=dict(T=1, D=1)): 0.95,  # TP: True Positive rate
        Assignment(values=dict(T=2, D=1)): 0.05,  # FT: False Negative rate
        
        Assignment(values=dict(T=1, D=2)): 0.1,   # FN: False Positive rate
        Assignment(values=dict(T=2, D=2)): 0.9,   # TN: True Negative rate
    },
    
    'P(S|D)': {
        Assignment(values=dict(S=1, D=1)): 0.8,
        Assignment(values=dict(S=2, D=1)): 0.2,
        Assignment(values=dict(S=1, D=2)): 0.3,
        Assignment(values=dict(S=2, D=2)): 0.7,
    }
}

for k, v in var.items():
    print(f'{k}:')
    display(v)

P(D):


{Assignment({'D': 1}): 0.01, Assignment({'D': 2}): 0.99}

P(T|D):


{Assignment({'T': 1, 'D': 1}): 0.95,
 Assignment({'T': 2, 'D': 1}): 0.05,
 Assignment({'T': 1, 'D': 2}): 0.1,
 Assignment({'T': 2, 'D': 2}): 0.9}

P(S|D):


{Assignment({'S': 1, 'D': 1}): 0.8,
 Assignment({'S': 2, 'D': 1}): 0.2,
 Assignment({'S': 1, 'D': 2}): 0.3,
 Assignment({'S': 2, 'D': 2}): 0.7}

# 4. Construct the Bayesian Network
The Bayesian Network consists of the defined factors. No explicit graph structure is needed for this task since we're focusing on computations.

In [41]:
factor_d = Factor(variables=[D], table=var['P(D)'])
factor_t_d = Factor(variables=[D, T], table=var['P(T|D)'])
factor_s_d = Factor(variables=[D, S], table=var['P(S|D)'])

display(factor_d)
display(factor_t_d)
display(factor_s_d)

Factor(variables=['D'], table=
(Assignment({'D': 1}), 0.01)
(Assignment({'D': 2}), 0.99))

Factor(variables=['D', 'T'], table=
(Assignment({'T': 1, 'D': 1}), 0.95)
(Assignment({'T': 2, 'D': 1}), 0.05)
(Assignment({'T': 1, 'D': 2}), 0.1)
(Assignment({'T': 2, 'D': 2}), 0.9))

Factor(variables=['D', 'S'], table=
(Assignment({'S': 1, 'D': 1}), 0.8)
(Assignment({'S': 2, 'D': 1}), 0.2)
(Assignment({'S': 1, 'D': 2}), 0.3)
(Assignment({'S': 2, 'D': 2}), 0.7))

# 5. Perform Inference


In [44]:
var['P(D|T)'] = factor_t_d * factor_d

# P(D | T=1)
table_f_d_t_positive = {
    assignment: prob for assignment, prob in var['P(D|T)'].table.items() if assignment.values["T"] == 1
}

display(table_f_d_t_positive)

f_d_t_positive = Factor(variables=[D], table=table_f_d_t_positive)
print('-' * 100)
display(f_d_t_positive)
print('-' * 100)
f_d_t_positive.normalize()
display(f_d_t_positive)

{Assignment({'D': 1, 'T': 1}): 0.0095, Assignment({'D': 2, 'T': 1}): 0.099}

----------------------------------------------------------------------------------------------------


Factor(variables=['D'], table=
(Assignment({'D': 1, 'T': 1}), 0.0095)
(Assignment({'D': 2, 'T': 1}), 0.099))

----------------------------------------------------------------------------------------------------


Factor(variables=['D'], table=
(Assignment({'D': 1, 'T': 1}), 0.08755760368663594)
(Assignment({'D': 2, 'T': 1}), 0.9124423963133641))

# 6. Analyze the Results

Analysis Questions

1.Interpretation: What does the posterior probability tell us about the patient's condition after observing a positive test result?

OK, Let's calculate it:
P(A|B) where A <-> "I'm ill" and B <-> 'test is positive'. We know that P(B = 1|A = 1) =  0.95, P(A) = 0.05, so P(B) = P(A) \cdot P(B | "ill" + (1 - P(A)) \cdot P(B | 'not ill') = 0.05 \cdot 0.95 + 0.95 \cdot 0.5 = 0.095. So, P(A|B) = P(A) \cdot \fraq{P(B|A), P(B)} = 0.05 \cdot 0.95 / 0.095 = 0.5 

2.Impact of Additional Evidence: How does the presence of the symptom (S=1) change the posterior probability compared to only having the test result?

3.Decision Making: Based on the computed probabilities, what would you recommend about further medical action?

4.Relation to Decision Making Theory: How does this exercise illustrate critical concepts in Decision Making Theory?

5.Reinforcement Learning: Describe the possible ideas of usage such techniques in RL
